In [1]:
import numpy as np
from coverage import gen_sats, gen_times, camera_model, forecast_fovs, calculate_revisits, revisit_map
from datetime import datetime, timezone, timedelta
import dataclasses
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import branca
import folium

from skyfield.framelib import itrs
from landsat import Instrument, Platform, Scene

! mkdir tmp

mkdir: tmp: File exists


In [2]:
start_dt = datetime.fromisoformat(Scene.start_utc)
num_days = 2

tles = gen_sats(
    sat_nos=[Platform.norad_id] # How to best handle multiple platforms? (TLE vs. SPG4 model too)
    # sat_nos=[39084]
    # sat_nos=[39084,49260]
)

inst = camera_model(
    name=Instrument.name, 
    fl=Instrument.focal_length_mm, 
    pitch=Instrument.pitch_um*1e-3, 
    h_pix=Instrument.rows, 
    v_pix=Instrument.cols, 
)

times = gen_times(
    start_yr=start_dt.year,
    start_mo=start_dt.month, 
    start_day=start_dt.day, 
    days=num_days, 
    step_min=Instrument.img_period)

xcell_size = ycell_size = .1

Satellite(s) Loaded from TLE:

[<EarthSatellite LANDSAT 9 catalog #49260 epoch 2022-06-15 11:12:37 UTC>]

Propogation time: 
 2022-07-26 00:00:00+00:00 
to 
 2022-07-27 23:59:48+00:00

In [3]:
## Batch FOV generation over N satellites - TODO: build multiple sats into config/ main script
gdfs = []
for tle in tles:
    sat = tle[0]

    fov_df = forecast_fovs(sat, times, inst)
    xyz_dist_rates = sat.at(times).frame_xyz_and_velocity(itrs)
    fov_df['x_pos'], fov_df['y_pos'], fov_df['z_pos'] = xyz_dist_rates[0].km
    fov_df['x_vel'], fov_df['y_vel'], fov_df['z_vel'] = xyz_dist_rates[1].km_per_s
    gdfs.append(fov_df)

fov_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="EPSG:4326")

In [4]:
fov_df["frac_days"] = fov_df.datetime.dt.dayofweek + fov_df.datetime.dt.hour/24 + fov_df.datetime.dt.minute/(24*60) + fov_df.datetime.dt.second/(24*60*60)
fov_df['time_gap'] = fov_df['frac_days'] - fov_df['frac_days'].shift(1)
fov_df[["datetime", "satellite", "id", "time", "geometry"]].dropna()

,datetime,satellite,id,time,geometry
0,2022-07-26 00:00:00+00:00,LANDSAT 9,149260,2022-07-26 00:00:00 UTC,"POLYGON ((-137.47666 80.39874, -141.45822 81.9..."
1,2022-07-26 00:00:22+00:00,LANDSAT 9,149260,2022-07-26 00:00:22 UTC,"POLYGON ((-144.66490 79.85927, -149.55689 81.2..."
2,2022-07-26 00:00:44+00:00,LANDSAT 9,149260,2022-07-26 00:00:44 UTC,"POLYGON ((-151.05964 79.18334, -156.47537 80.5..."
3,2022-07-26 00:01:06+00:00,LANDSAT 9,149260,2022-07-26 00:01:06 UTC,"POLYGON ((-156.66449 78.39473, -162.30947 79.6..."
4,2022-07-26 00:01:28+00:00,LANDSAT 9,149260,2022-07-26 00:01:28 UTC,"POLYGON ((-161.53933 77.51473, -167.21320 78.6..."
...,...,...,...,...,...
7850,2022-07-27 23:58:20+00:00,LANDSAT 9,149260,2022-07-27 23:58:20 UTC,"POLYGON ((163.56990 46.82996, 161.26742 47.245..."
7851,2022-07-27 23:58:42+00:00,LANDSAT 9,149260,2022-07-27 23:58:42 UTC,"POLYGON ((163.05234 45.52560, 160.80283 45.934..."
7852,2022-07-27 23:59:04+00:00,LANDSAT 9,149260,2022-07-27 23:59:04 UTC,"POLYGON ((162.55460 44.21930, 160.35475 44.622..."
7853,2022-07-27 23:59:26+00:00,LANDSAT 9,149260,2022-07-27 23:59:26 UTC,"POLYGON ((162.07499 42.91118, 159.92174 43.308..."


In [5]:
# %%timeit
# get_inst_fov(sat, times[0], inst)

In [6]:
## Set ascending/ descending direction
conditions = [
    fov_df['z_vel'].lt(0),
    fov_df['z_vel'].gt(0)
]

choices = ['dsc','asc']
fov_df['asc_dsc'] = np.select(conditions, choices, default='undefined')

## Drop ascending pass FOVs
fov_df.loc[fov_df.asc_dsc=="asc", "geometry"] = None

In [7]:
## Select AOI from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Drop Antarctica
world = world[world.continent != "Antarctica"]

## Drop any FOVs not over land area
join = fov_df.sjoin(world, how="left").dropna()
fov_df.loc[join.index, 'mode'] = "SCIENCE"
fov_df.loc[~fov_df.index.isin(join.index), 'mode'] = "STANDBY"
fov_df.loc[fov_df['mode']=="STANDBY", "geometry"] = None

In [8]:
## Plotting FOVs

## Create cmap for unique satellites and create color column
## TODO: Move to plotting function?
sat_ids = list(fov_df["id"].unique()).sort()
cmap = branca.colormap.StepColormap(['red', 'blue'], sat_ids, vmin=139084, vmax = 149260)
fov_df['color'] = fov_df['id'].apply(cmap)

## Make a folium map
m = fov_df[fov_df['mode']=="SCIENCE"].drop('datetime', axis=1).explore(color="color", style_kwds={'fillOpacity':0.2}, tooltip=["satellite", "time"])

## View or save
m#.save("./tmp/fovs_map.html")

In [9]:
## Coverage data analysis for single satellite/ batch of satellites

## Set AOI
# aoi =  world[world.name == "Brazil"].geometry
# aoi =  world[world.continent == "North America"].geometry
# aoi = world[world.name == "United States of America"].geometry # Includes Alaska...

## Or read in aoi from .geojson
aoi = gpd.read_file('./aois/eastern_us.geojson').geometry # ...so use AOI for subsection of US

## Filter fov_df by aoi
xmin, ymin, xmax, ymax= aoi.total_bounds
revisit_df = fov_df.cx[xmin: xmax, ymin: ymax]

## Create revisit map on regular grid
grid, grid_shape = calculate_revisits(revisit_df, aoi, grid_x=xcell_size, grid_y=ycell_size)
grid.n_visits.fillna(0).describe()
m = revisit_map(grid, grid_shape, grid_x=xcell_size, grid_y=ycell_size)

# Add WRS2 - Only use with AOI applied!
wrs2 = gpd.read_file('./WRS2_descending_0/WRS2_descending.shp')
wrs2 = wrs2.cx[xmin: xmax, ymin: ymax]
folium.GeoJson(data=wrs2["geometry"], overlay=False).add_to(m)

m#.save("./tmp/revisits_map.html")

In [10]:
# TODO: Fix saves to GeoJSON (done using patch below)
import fiona

# grid.to_file('./tmp/all_revisits.geojson')

save_df = fov_df.loc[:,['satellite', 'geometry', 'time']].dropna()
save_df['satellite'] = save_df["satellite"].astype(str)
save_df['time'] = save_df["time"].astype(str)

## Save to geojson based on sat name
for satname in save_df.satellite.unique():
    with fiona.Env(OSR_WKT_FORMAT="WKT2_2018"):
        save_df[save_df.satellite==satname].to_file("./tmp/{}_fovs.geojson".format(satname.replace(" ", "_")), engine="pyogrio")

/Users/kevinlalli/miniconda3/envs/coverage/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
